In [6]:
from glob import glob

ckpoints = glob("outs/*")
list(enumerate(ckpoints))

[(0, 'outs/200112_with_dp'),
 (1, 'outs/200113_with_dp'),
 (2, 'outs/test_city'),
 (3, 'outs/test_wpef')]

In [22]:
idx = 1

from configs.parser import Parser

C = Parser(f"{ckpoints[idx]}/config.yaml").C
C

data:
  dataset:
    mode: fine
    root: /data2/DW/point-semantic/cityscapes
    target_type: semantic
  loader:
    batch_size: 32
  name: cityscapes
net:
  deeplab:
    head_in_ch: 2048
    num_classes: 20
    pretrained: false
    resnet: res103
  pointhead:
    beta: 0.75
    in_c: 533
    k: 3
    num_classes: 20
run:
  epochs: 200

In [23]:
states = glob(f"{ckpoints[idx]}/*.pth")
list(enumerate(states))

[(0, 'outs/200113_with_dp/epoch_0000_loss_1.00507.pth'),
 (1, 'outs/200113_with_dp/epoch_0010_loss_0.49184.pth'),
 (2, 'outs/200113_with_dp/epoch_0020_loss_0.47258.pth'),
 (3, 'outs/200113_with_dp/epoch_0030_loss_0.44325.pth'),
 (4, 'outs/200113_with_dp/epoch_0040_loss_0.38861.pth'),
 (5, 'outs/200113_with_dp/epoch_0050_loss_0.31817.pth'),
 (6, 'outs/200113_with_dp/epoch_0060_loss_0.28252.pth'),
 (7, 'outs/200113_with_dp/epoch_0070_loss_0.23913.pth'),
 (8, 'outs/200113_with_dp/epoch_0080_loss_0.21490.pth'),
 (9, 'outs/200113_with_dp/epoch_0090_loss_0.17623.pth'),
 (10, 'outs/200113_with_dp/epoch_0100_loss_0.13323.pth'),
 (11, 'outs/200113_with_dp/epoch_0110_loss_0.09992.pth'),
 (12, 'outs/200113_with_dp/epoch_0120_loss_0.08335.pth'),
 (13, 'outs/200113_with_dp/epoch_0130_loss_0.06446.pth'),
 (14, 'outs/200113_with_dp/epoch_0140_loss_0.05860.pth'),
 (15, 'outs/200113_with_dp/epoch_0150_loss_0.03984.pth'),
 (16, 'outs/200113_with_dp/epoch_0160_loss_0.03032.pth'),
 (17, 'outs/200113_with_

In [25]:
import torch
from model import deeplabv3, PointHead, PointRend

idx = -1 
state = torch.load(f"{states[idx]}")

pointrend = PointRend(
    deeplabv3(**C.net.deeplab),
    PointHead(**C.net.pointhead)
)
pointhead = torch.nn.DataParallel(pointrend)
# pointrend.load_state_dict(state)

pointrend = pointhead.module
print(f"Load Done {states[idx]}")

Load Done outs/200113_with_dp/epoch_0290_loss_0.02130.pth


In [ ]:

@torch.no_grad()
def step(net, loader):
    net.eval()
 
    for i, (x, gt) in enumerate(loader):

        